In [1]:
import mujoco
import gymnasium as gym
import numpy as np
import torch
import matplotlib.pyplot as plt
%env MUJOCO_GL=egl

env: MUJOCO_GL=egl


In [2]:
xml = """
<mujoco>
  <worldbody>
    <geom name="red_box" type="box" size=".2 .2 .2" rgba="1 0 0 1"/>
    <geom name="green_sphere" pos=".2 .2 .2" size=".1" rgba="0 1 0 1"/>
  </worldbody>
</mujoco>
"""
model = mujoco.MjModel.from_xml_string(xml)

In [4]:
data = mujoco.MjData(model)

In [5]:
print(data.geom_xpos)

[[0. 0. 0.]
 [0. 0. 0.]]


the position of the green sphere is supposed to be offset from the origin, but it is not. We have to propagate the kinematics to get the correct position:

In [6]:
mujoco.mj_kinematics(model, data)
print('raw access:\n', data.geom_xpos)

# MjData also supports named access:
print('\nnamed access:\n', data.geom('green_sphere').xpos)

raw access:
 [[0.  0.  0. ]
 [0.2 0.2 0.2]]

named access:
 [0.2 0.2 0.2]


In [11]:
import os
os.environ["DISPLAY"] = "localhost:10.0"

In [2]:
xml = """
<mujoco>
  <worldbody>
    <geom name="red_box" type="box" size=".2 .2 .2" rgba="1 0 0 1"/>
    <geom name="green_sphere" pos=".2 .2 .2" size=".1" rgba="0 1 0 1"/>
  </worldbody>
</mujoco>
"""
# Make model and data
model = mujoco.MjModel.from_xml_string(xml)
data = mujoco.MjData(model)

# Make renderer, render and show the pixels
with mujoco.Renderer(model) as renderer:
  media.show_image(renderer.render())

/home/michael/anaconda3/envs/mujoco/lib/python3.12/site-packages/glfw/__init__.py:914: GLFWError: (65544) b'X11: Failed to open display localhost:10.0'
  warnings.warn(message, GLFWError)
/home/michael/anaconda3/envs/mujoco/lib/python3.12/site-packages/glfw/__init__.py:914: GLFWError: (65537) b'The GLFW library is not initialized'
  warnings.warn(message, GLFWError)


FatalError: an OpenGL platform library has not been loaded into this process, this most likely means that a valid OpenGL context has not been created before mjr_makeContext was called